In [2]:
import pandas as pd

In [36]:
df = pd.read_json('static/events.json', orient='columns')


In [22]:
datadf = df['events'].apply(pd.Series)

In [23]:
datadf['md'] = datadf['date'].str.split('-', n=1).str[-1]

In [25]:
datadf['out'] = datadf['date']+'：'+datadf['event']

In [42]:
res_s = datadf.groupby('md')['out'].agg(list).reset_index(drop=False)
res_s

,md,out
0,01-01,"[1942-01-01：26个盟国签署《联合国家宣言》, 1945-01-01：德国从阿登撤退]"
1,01-02,[1943-01-02：德国开始从高加索撤退]
2,01-06,[1944-01-06：苏联军队进入波兰]
3,01-08,[1940-01-08：英国开始实行配给制]
4,01-10,[1943-01-10：苏联开始对斯大林格勒的德国军队发起攻势]
...,...,...
199,12-24,[1943-12-24：苏联在乌克兰前线发起攻势]
200,12-26,[1944-12-26：巴顿解救巴斯托涅]
201,12-27,[1944-12-27：苏联军队围攻布达佩斯]
202,12-29,[1940-12-29：德国对伦敦进行大规模空袭]


In [53]:
date_s = pd.date_range('2024-01-01',periods=366, freq='d')
res_df = pd.DataFrame(date_s.strftime('%m-%d'), columns=['sdate'])

In [54]:
res_df= res_df.merge(res_s, left_on='sdate', right_on='md', how='outer')

In [65]:
res_df['mon'] = res_df['sdate'].str[:2]
res_df['day'] = res_df['sdate'].str[3:]

In [73]:
res_df['out'] = res_df[['sdate', 'out']].to_dict('records')

In [75]:
res_tb = res_df.pivot_table(index='mon', columns='day', values='out', aggfunc='first')
res_tb.to_json('test.json', orient='index', force_ascii=False, indent=True)

In [52]:
res_df

,sdate,mon,day
0,01-01,01,1
1,01-02,01,2
2,01-03,01,3
3,01-04,01,4
4,01-05,01,5
...,...,...,...
361,12-27,12,7
362,12-28,12,8
363,12-29,12,9
364,12-30,12,0


In [47]:
res_tb

sdate                                                               
day      0      1      2      3      4      5      6      7      8      9
mon                                                                      
01   01-10  01-01  01-02  01-03  01-04  01-05  01-06  01-07  01-08  01-09
02   02-10  02-01  02-02  02-03  02-04  02-05  02-06  02-07  02-08  02-09
03   03-10  03-01  03-02  03-03  03-04  03-05  03-06  03-07  03-08  03-09
04   04-10  04-01  04-02  04-03  04-04  04-05  04-06  04-07  04-08  04-09
05   05-10  05-01  05-02  05-03  05-04  05-05  05-06  05-07  05-08  05-09
06   06-10  06-01  06-02  06-03  06-04  06-05  06-06  06-07  06-08  06-09
07   07-10  07-01  07-02  07-03  07-04  07-05  07-06  07-07  07-08  07-09
08   08-10  08-01  08-02  08-03  08-04  08-05  08-06  08-07  08-08  08-09
09   09-10  09-01  09-02  09-03  09-04  09-05  09-06  09-07  09-08  09-09
10   10-10  10-01  10-02  10-03  10-04  10-05  10-06  10-07  10-08  10-09
11   11-10  11-01  11-02  11-03  11-04  11-05  11-06  11-07  11-08  11-09
12   12-10  12-01  12-02  12-03  12-04  12-05  12-06  12-07  12-08  12-09

In [18]:
dfe = pd.json_normalize(df['events'])

In [2]:
def extract_secondary_headings(markdown_file):
    secondary_headings = []
    
    with open(markdown_file, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith('## '):
                line = line.strip()
                line = line.rsplit(' ', 1)[-1]
                secondary_headings.append(line)
    
    return secondary_headings

def check_missing_date(mydate):
    daterange = pd.date_range(start='1939-01-01', end='1945-12-31',freq='D')
    dfr = pd.DataFrame(daterange, columns=['dr'])
    dateset = set(dfr['dr'].dt.date.apply(str))
    return dateset - set(mydate)

In [5]:
mydate = extract_secondary_headings('ww2cn.md')

In [6]:
len(check_missing_date(mydate))
check_missing_date(mydate)

set()

In [11]:
import re
from pathlib import Path

def split_markdown_file(markdown_file, folder):
    dir = Path(folder)
    dir.mkdir(exist_ok=True)
    with open(markdown_file, 'r', encoding='utf-8') as file:
        content = file.read()

    # 删除所有一级标签
    content = re.sub(r'^#\s.*$', '', content, flags=re.MULTILINE)

    # 以二级标签为特征分割文件
    sections = re.split(r'^##\s', content, flags=re.MULTILINE)

    # 去掉第一个空元素
    if sections[0].strip() == '':
        sections = sections[1:]

    # 保存每个分割后的部分为单独的文件
    for section in sections:
        # 提取二级标签内容作为文件名
        heading = re.match(r'(.*?)\n', section)
        if heading:
            file_name = heading.group(1).strip().replace(' ', '_') + '.md'
            section = '# ' + section
            # 将二级标题提升为一级标题，三级标题提升为二级标题
            section = re.sub(r'^##\s', '# ', section, flags=re.MULTILINE)
            section = re.sub(r'^###\s', '## ', section, flags=re.MULTILINE)

            with open(dir/file_name, 'w', encoding='utf-8') as output_file:
                output_file.write(section)
            # print(f"Saved section to {file_name}")


In [12]:
split_markdown_file('ww2cn.md', 'ww2cnbyday')


In [12]:
import json
from datetime import datetime, timedelta

def generate_date_json(file_path, events_file_path):
    start_date = datetime(1939, 1, 1)
    end_date = datetime(1945, 12, 31)
    ref_date = datetime(1939, 9, 1)
    date_json = {}

    # 读取 events.json 文件
    with open(events_file_path, 'r', encoding='utf-8') as events_file:
        events_data = json.load(events_file)
        events = events_data.get('events', [])

    current_date = start_date
    while current_date <= end_date:
        year = current_date.year
        month = current_date.strftime('%m')
        day = current_date.strftime('%d')
        days_to = (current_date - ref_date).days
        date_str = current_date.strftime('%Y-%m-%d')

        if year not in date_json:
            date_json[year] = {}
        if month not in date_json[year]:
            date_json[year][month] = []

        # 查找当前日期的事件
        event_info = next((event for event in events if event['date'] == date_str), None)
        if event_info:
            date_json[year][month].append({
                'date': date_str,
                'days_to': days_to,
                'event': event_info['event']
            })
        else:
            date_json[year][month].append({
                'date': date_str,
                'days_to': days_to
            })

        current_date += timedelta(days=1)

    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(date_json, file, ensure_ascii=False, indent=4)

In [13]:
generate_date_json('mydate.json', './static/events.json')